In [1]:
# Setup
# `jupyter nbextension enable --py --sys-prefix ipyvuetify`#
# start: `voila .\smart_grids.ipynb`
# icons here https://vuetifyjs.com/en/features/icon-fonts/#material-design-icons


In [2]:
import ipywidgets
from matplotlib import pyplot as plt
import numpy as np
import ipyvuetify as v
from util.get_photovoltaic_data import get_photovoltaic_data
from loguru import logger
from datetime import datetime
from tqdm import tqdm
import result_fig
import pandas as pd
import numpy as np


In [3]:
def update_stepper(widget, *args):
    stepper.v_model += widget.direction
    try:
        widget.step.complete = True
    except:
        pass


In [4]:
class BackBtn(v.Btn):
    def __init__(self):
        self.direction = -1
        super().__init__(children="Back")
        self.on_event("click", update_stepper)


In [5]:
class NextBtn(v.Btn):
    def __init__(self, container, step, disabled=True):
        self.step = step
        self.direction = 1
        self.container = container
        container.next_btn = self
        super().__init__(children="Next")
        self.on_event("click", update_stepper)
        self.disabled = disabled


In [6]:
import address_widget

address_container = address_widget.Address()
# address_container
address_step_number = 1
address_step = v.StepperStep(
    step=address_step_number, children=["Address"], complete=False
)
address_step_content = v.StepperContent(
    step=address_step_number,
    children=[address_container, NextBtn(address_container, address_step)],
)


In [7]:
import battery_widget

battery_container = battery_widget.Battery()
# battery_container
battery_step_number = 2
battery_step = v.StepperStep(
    step=battery_step_number, children=["Battery"], complete=False
)
battery_step_content = v.StepperContent(
    step=battery_step_number,
    children=[battery_container, BackBtn(), NextBtn(battery_container, battery_step)],
)


In [8]:


def convert_pv_data_to_dataframe(data):
    dict = {"date": [], "production":[]}  # "date": [],

    for d in data:
        thedate= datetime.strptime(d["time"],"%Y%m%d:%H%M")
        dict["date"].append(thedate.strftime("%Y-%m-%d %H"))
        dict["production"].append(d["PV system power"])

    return pd.DataFrame(dict)


In [9]:
def get_pv_data():
    response = get_photovoltaic_data(
        latitude=address_container.latitude.v_model,
        longitude=address_container.longitude.v_model,
        pv_tech=pv_container.pv_technology.v_model,
        pv_peak_power=pv_container.pv_peak_power.v_model,
        pv_loss=pv_container.pv_loss.v_model,
        pv_angle=pv_container.pv_slope.v_model,
        pv_azimuth=pv_container.pv_azimuth.v_model,
        start_year=2015,
        end_year=2015,
    )
    pv_data = [
        {"time": data["time"], "PV system power": data["P"]}
        for data in response["outputs"]["hourly"]
    ]
    return convert_pv_data_to_dataframe(pv_data)

In [10]:
import pv_widget

pv_container = pv_widget.Photovoltaic()
pv_step_number = 3
pv_step = v.StepperStep(step=pv_step_number, children=["Photvoltaic"], complete=False)
pv_calc_btn = v.Btn(children=["calculate"],disabled=True)
pv_container.calc_btn=pv_calc_btn
pv_next_button=NextBtn(pv_container, pv_step)

def calculate_pv_power(widget, *args):
    pv_container.pv_card.loading = True
    pv_container.pv_power_data = get_pv_data()
    pv_step.complete = True
    pv_container.children=[pv_container.pv_card]+[pv_container.get_fig()]
    pv_next_button.disabled=False

    pv_container.pv_card.loading = False

pv_calc_btn.on_event("click", calculate_pv_power)
pv_step_content = v.StepperContent(
    step=pv_step_number, children=[pv_container, BackBtn(),pv_calc_btn, pv_next_button]
)


In [11]:
# pv_power_container = v.Container(children=[get_data_fig()])
# pv_power_step_number = 4
# pv_power_step = v.StepperStep(
#     step=pv_power_step_number, children=["PV Power"], complete=False
# )
# pv_power_step_content = v.StepperContent(
#     step=pv_power_step_number,
#     children=[
#         pv_power_container,
#         BackBtn(),
#         NextBtn(pv_power_container, pv_power_step, disabled=False),
#     ],
# )


In [12]:
# import consumers_widget

# consumer_container = consumers_widget.ConsumerTable()

# consumer_step_number = 5 # change
# consumer_step = v.StepperStep(
#     step=consumer_step_number, children=["Consumers"], complete=False
# )
# consumer_step_content = v.StepperContent(
#     step=consumer_step_number,
#     children=[
#         consumer_container,
#         BackBtn(),
#         NextBtn(consumer_container, consumer_step,disabled=False),
#     ],
# )


In [13]:
import usage_widget
usage_container = usage_widget.Usage()

usage_step_number = 4
usage_step = v.StepperStep(
    step=usage_step_number, children=["Consumers"], complete=False
)
usage_next_btn=NextBtn(usage_container, usage_step)
def usage_next_btn_event(widget, *args):
    stepper.v_model += widget.direction
    try:
        widget.step.complete = True
    except:
        pass
    result_container.load()
usage_container.next_btn=usage_next_btn
usage_next_btn.on_event("click",usage_next_btn_event)
usage_step_content = v.StepperContent(
    step=usage_step_number,
    children=[
        usage_container,
        BackBtn(),
        usage_next_btn,
    ],
)

In [14]:
# import importlib
# importlib.reload(result_widget)
import result_widget
result_container = result_widget.Result(
    battery_container,
    pv_container,    usage_container)

result_step_number = 5
result_step = v.StepperStep(
    step=result_step_number, children=["Result"], complete=False
)
result_step_content = v.StepperContent(
    step=result_step_number,
    children=[
        result_container,
        BackBtn(),
    ],
)

In [15]:
stepper = v.Stepper(
    v_model=1,# TODO change to 1
    children=[
        v.StepperHeader(
            children=[
                address_step,
                v.Divider(),
                battery_step,
                v.Divider(),
                pv_step,
                v.Divider(),
                usage_step,
                v.Divider(),
                result_step
            ]
        ),
        v.StepperItems(
            children=[
                address_step_content,
                battery_step_content,
                pv_step_content,
                usage_step_content,
                result_step_content
            ]
        ),
    ],
)
stepper


Stepper(children=[StepperHeader(children=[StepperStep(children=['Address'], complete=False, step=1.0), Divider…